# Data Preprocsesing

In [18]:
# import json

In [19]:
'''
with open('data/trec_corpus_20220301_plain.json') as read_from_file:
    with open('data/preprocessed_corputs.csv', 'a') as write_to_file:
        
        # write header of csv file
        write_to_file.write('id, text')
        doc = read_from_file.readline()

        for i in range(5): # while doc:
            
            # parse json
            doc_parsed = json.loads(doc)
            doc_text = doc_parsed['title'] + ' ' + doc_parsed['plain']
            
            print(doc_parsed)
            print('\n\n############\n\n')

            # do preprocessing
            preprocessed_text = 'test test test test test'

            # write preprocessed document to file (append)
            write_to_file.write(f'{doc_id}, {preprocessed_text}\n')

            # read next line
            doc = read_from_file.readline()
'''

"\nwith open('data/trec_corpus_20220301_plain.json') as read_from_file:\n    with open('data/preprocessed_corputs.csv', 'a') as write_to_file:\n        \n        # write header of csv file\n        write_to_file.write('id, text')\n        doc = read_from_file.readline()\n\n        for i in range(5): # while doc:\n            \n            # parse json\n            doc_parsed = json.loads(doc)\n            doc_text = doc_parsed['title'] + ' ' + doc_parsed['plain']\n            \n            print(doc_parsed)\n            print('\n\n############\n\n')\n\n            # do preprocessing\n            preprocessed_text = 'test test test test test'\n\n            # write preprocessed document to file (append)\n            write_to_file.write(f'{doc_id}, {preprocessed_text}\n')\n\n            # read next line\n            doc = read_from_file.readline()\n"

# Approach of Abda
- Loading data
- Preprocessing first 5 lines to save computation
- Appending preprecessed data into a dictionary / dataframe 
- Saving finished data after preprocessing into csv or feather file for continuous analysis and evaluation



In [20]:
# Loading necessary libraries
import pandas as pd
import nltk

In [36]:
# Loading dataset -> Only first five rows
df_prep = pd.read_json('/Users/abderrahmanecharrade/Desktop/Uni-Mannheim-Sonder/04_Semester/IRWS/Data/trec_corpus_20220301_plain.json', lines=True)

# Dropping url-column
df_prep.drop(columns=['url'], inplace=True, axis=1)
display(df_prep.head())

# Looking for missing / empty values
print('\nCheck first stats of first 10 rows')
print('------------------------------------')
print(df_prep.isna().sum().sort_values)

# Concatenating title column with plain-text column
df_prep['plain-text'] = df_prep['title'].astype(str) + ' ' + df_prep['plain'].astype(str)
df_prep.drop(['title', 'plain'], axis = 1, inplace = True)
df_prep.rename(columns = {'plain-text' : 'plain'}, inplace = True)
display(df_prep.head())

## __Cleaning and preprocessing textual data__
- Converting to lower case
- How to handle missing data (No necessaty...?)
- Removing punctuations
- Removing stopwords
- Tokenizing data (especially last column with plain text)
- Normalizing data (Stemming / Lemmatization)



In [22]:
# ========= Lower case whole column =========
df_prep['plain'] = df_prep['plain'].str.lower()
df_prep.sample(frac = 1).head()

,id,plain
98,49647788,białe augustowskie (lake) białe augustowskie (...
29,56457702,franca ronchese franca ronchese is an italian-...
46,24522576,irving corey second lieutenant irving banfield...
38,65057452,1896 tipperary senior hurling championship the...
4,13924699,william clark (inventor) william clark (17 mar...


In [25]:
# ========= Tokenization =========
from nltk import word_tokenize
test_doc = df_prep.iloc[0,1]
test_doc_tokens = word_tokenize(test_doc)
print(test_doc_tokens)

def tokenize_words(plain_text):
    tokenized_text = word_tokenize(plain_text)
    return tokenized_text

df_prep_tokenized = df_prep['plain'].apply(lambda x: tokenize_words(x))


['keith', 'osik', 'keith', 'richard', 'osik', '(', 'born', 'october', '22', ',', '1968', ')', ',', 'is', 'a', 'former', 'major', 'league', 'baseball', 'catcher', 'who', 'played', 'in', 'the', 'major', 'leagues', 'from', '-', '.', 'he', 'played', 'for', 'the', 'milwaukee', 'brewers', ',', 'pittsburgh', 'pirates', ',', 'baltimore', 'orioles', ',', 'and', 'washington', 'nationals', '.', 'he', 'was', 'drafted', 'in', 'the', '24th', 'round', 'of', 'the', 'mlb', 'draft', '.', 'his', 'brother', 'was', 'also', 'a', 'professional', 'baseball', 'player', ',', 'but', 'only', 'played', 'in', 'the', 'minors', '.', 'he', 'was', 'born', 'in', 'port', 'washington', ',', 'new', 'york', ',', 'but', 'now', 'lives', 'in', 'shoreham', ',', 'new', 'york', '.', 'osik', 'is', 'currently', 'the', 'head', 'baseball', 'coach', 'at', 'farmingdale', 'state', 'college', ',', 'a', 'division', 'iii', 'institution', 'located', 'on', 'long', 'island', 'in', 'new', 'york', '.', 'he', 'was', 'inducted', 'into', 'the', 's

In [26]:
# ========= Removing punctuations =========
import string
def remove_punctuations(plain_text):
    punctiations = string.punctuation
    return plain_text.translate(str.maketrans('', '', punctiations))

df_prep['plain'] = df_prep['plain'].apply(lambda x : remove_punctuations(x))

In [27]:
# ========= Removing stopwords =========
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))


def remove_stopwords(plain_text):
    return ' '.join([word for word in plain_text.split() if word not in STOPWORDS])

df_prep['plain'] = df_prep['plain'].apply(lambda x: remove_stopwords(x))

In [31]:
# ========= Removing special characters =========
import re

def remove_spec_char(plain_text):
    plain_text = re.sub('[^a-zA-Z0-9]', ' ', plain_text)
    plain_text = re.sub('\s+', ' ', plain_text)
    return plain_text

df_prep['plain'] = df_prep['plain'].apply(lambda x : remove_spec_char(x))

In [ ]:
'''
# ========= Stemming =========
# For the Stemming we create a separate series which will append the last feature of the dataframe
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def stem_words(plain_text):
    return ' '.join([ps.stem(word) for word in plain_text.split()])

df_prep['plain_stemmed'] = df_prep['plain'].apply(lambda x : stem_words(x))
'''


### Lemmatization instead of Stemming:
> "Stemming is a faster process than lemmatization as stemming chops off the word irrespective of the context, whereas the latter is context-dependent. Stemming is a rule-based approach, whereas lemmatization is a canonical dictionary-based approach. Lemmatization has higher accuracy than stemming."<br></br>
     [__Explained Lemmatization vs. Stemming__]("https://analyticsindiamag.com/explained-stemming-vs-lemmatization-in-nlp/")<br></br>
We have decided to move further with Lemmatization, because some words inspected by sampling were not representing the context and meaning of possibly queried terms

In [35]:
# ========= Lemmatization =========
# For the Lemmatization we create a separate series which will append the last feature of the dataframe
from nltk import pos_tag
from nltk.corpus import wordnet
# Comment two lines below out if not packages 'wordnet' & 'omw-1.4' not available 
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {'N':wordnet.NOUN, 'V':wordnet.VERB, 'J':wordnet.ADJ, 'R':wordnet.ADV}

def lemmatize_word(plain_text):
    # Finind pos tags
    pos_text = pos_tag(plain_text.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_text])

df_prep['plain_lemmatized'] = df_prep['plain'].apply(lambda x : lemmatize_word(x))

# __Finished preprocessing__
If the preprocessing fulfills the requirements for further operations and building IR-Models,
export finished dataframe as csv or [__feather__]('https://arrow.apache.org/docs/python/feather.html') file format (light-weighted option to csv) which saves in comparison to json and csv more computation power and cpu time<br></br>
Next task is to build up the vector space model and starting with tf-idf / term-weighting

----------------------------------------------------------------
## __Writing in feather file format__
import pyarrow.feather as feather<br>
feather.write_feather(df, '/path/to/file')

## __Writing in csv file format__
DataFrame.to_csv('/path/to/file')<br></br>

If the size of the finished and exported file is still too large, add it to .gitignorefile<br>
Github for private use (non-commercial) does not allows too large repositories